In [ ]:
import sys
import os
from pathlib import Path

# Caminho absoluto para a raiz do projeto (um nível acima de notebooks)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)


In [ ]:
from config.utils import *


In [ ]:
teste = obter_dados_api(INDICADORES['IPCA'])

In [ ]:
#INDICADORES['IPCA']['URL'],
INDICADORES['DESEMPREGO']['URL']

In [ ]:
teste[0]['resultados'][0]['series'][0]['serie']

In [ ]:
salvar_dados_api(teste, 'ipca')

In [ ]:
for nome_indicador, config in INDICADORES.values():
    try:
        dados = obter_dados_api(indicador)
        salvar_dados_api(dados, indicador['NOME'])
    except Exception as e:
        logger.error(f"Erro ao processar o indicador {indicador['NOME']}: {str(e)}")

In [ ]:

Cotaçao cambio
response = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.21619/dados?formato=json&dataInicial=29/06/2015")
response.json()

In [ ]:
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json&dataInicial=01/07/2015&dataFinal=01/07/2025"

response = requests.get(url)
response.raise_for_status()  
dados_cambio = response.json()

In [ ]:
dados_cambio